# MyAnimeList API Data Extraction
Notebook for data pull from Jikan, an unofficial [MyAnimeList.net](https://myanimelist.net/) API.
- Unfortunately, MyAnimeList's official API is still in beta and lacks many features.
- Jikan operates by scraping the website.
- [Jikan API Documentation can be found here](https://docs.api.jikan.moe/).

### Importing Libraries:

In [2]:
import numpy as np
import pandas as pd
import requests
import time

from datetime import datetime
from sklearn.preprocessing import MultiLabelBinarizer

### 1. Jikan Unofficial API:
- Note: The API has a limit of 30 requests/minute and 2 requests/second.

In [14]:
BASE_URL = 'https://api.jikan.moe/v4'

SEASON_LIST_ENDPOINT = f'{BASE_URL}/seasons'

In [12]:
# Extract all seasons from API
response = requests.get(SEASON_LIST_ENDPOINT)
season_list = response.json()
season_list

{'pagination': {'last_visible_page': 1, 'has_next_page': False},
 'data': [{'year': 2025, 'seasons': ['winter']},
  {'year': 2024, 'seasons': ['winter', 'spring', 'summer', 'fall']},
  {'year': 2023, 'seasons': ['winter', 'spring', 'summer', 'fall']},
  {'year': 2022, 'seasons': ['winter', 'spring', 'summer', 'fall']},
  {'year': 2021, 'seasons': ['winter', 'spring', 'summer', 'fall']},
  {'year': 2020, 'seasons': ['winter', 'spring', 'summer', 'fall']},
  {'year': 2019, 'seasons': ['winter', 'spring', 'summer', 'fall']},
  {'year': 2018, 'seasons': ['winter', 'spring', 'summer', 'fall']},
  {'year': 2017, 'seasons': ['winter', 'spring', 'summer', 'fall']},
  {'year': 2016, 'seasons': ['winter', 'spring', 'summer', 'fall']},
  {'year': 2015, 'seasons': ['winter', 'spring', 'summer', 'fall']},
  {'year': 2014, 'seasons': ['winter', 'spring', 'summer', 'fall']},
  {'year': 2013, 'seasons': ['winter', 'spring', 'summer', 'fall']},
  {'year': 2012, 'seasons': ['winter', 'spring', 'summer',

In [13]:
# Transform into list of seasons
year_season_pairs = [(entry['year'], season) for entry in season_list['data'] for season in entry['seasons']]
year_season_pairs

[(2025, 'winter'),
 (2024, 'winter'),
 (2024, 'spring'),
 (2024, 'summer'),
 (2024, 'fall'),
 (2023, 'winter'),
 (2023, 'spring'),
 (2023, 'summer'),
 (2023, 'fall'),
 (2022, 'winter'),
 (2022, 'spring'),
 (2022, 'summer'),
 (2022, 'fall'),
 (2021, 'winter'),
 (2021, 'spring'),
 (2021, 'summer'),
 (2021, 'fall'),
 (2020, 'winter'),
 (2020, 'spring'),
 (2020, 'summer'),
 (2020, 'fall'),
 (2019, 'winter'),
 (2019, 'spring'),
 (2019, 'summer'),
 (2019, 'fall'),
 (2018, 'winter'),
 (2018, 'spring'),
 (2018, 'summer'),
 (2018, 'fall'),
 (2017, 'winter'),
 (2017, 'spring'),
 (2017, 'summer'),
 (2017, 'fall'),
 (2016, 'winter'),
 (2016, 'spring'),
 (2016, 'summer'),
 (2016, 'fall'),
 (2015, 'winter'),
 (2015, 'spring'),
 (2015, 'summer'),
 (2015, 'fall'),
 (2014, 'winter'),
 (2014, 'spring'),
 (2014, 'summer'),
 (2014, 'fall'),
 (2013, 'winter'),
 (2013, 'spring'),
 (2013, 'summer'),
 (2013, 'fall'),
 (2012, 'winter'),
 (2012, 'spring'),
 (2012, 'summer'),
 (2012, 'fall'),
 (2011, 'winter'),


In [157]:
# Extract all anime data by iterating through all pages for each season
def fetch_seasonal_anime(year, season):
    anime_list = []
    current_page = 1
    while True:
        season_url = f'{BASE_URL}/seasons/{year}/{season}?continuing=false&page={current_page}'
        try:
            response = requests.get(season_url)
            response.raise_for_status()  # Raise an exception for HTTP errors
            response_json = response.json()
            data = response_json.get('data', [])
            pagination = response_json.get('pagination', {})
            anime_list.extend(data)
            
            if not pagination.get('has_next_page', False):
                break

            current_page += 1
            
            time.sleep(1.5)  # Sleep for 1.5 seconds to stay within the rate limit

        except requests.RequestException as e:
            print(f"An error occurred: {e}")
            break

    return anime_list

def compile_seasonal_anime(year_season_list):
    all_anime_list = []
    for year, season in year_season_pairs:
        all_anime_list.extend(fetch_seasonal_anime(year, season))
        time.sleep(1.5)  # Sleep for 1.5 seconds to stay within the rate limit
    return all_anime_list

In [159]:
full_anime_list = compile_seasonal_anime(year_season_pairs)

In [8]:
# Check output
full_anime_list[6323]

{'mal_id': 36587,
 'url': 'https://myanimelist.net/anime/36587/Granblue_Fantasy_The_Animation_Season_2',
 'images': {'jpg': {'image_url': 'https://cdn.myanimelist.net/images/anime/1948/103308.jpg',
   'small_image_url': 'https://cdn.myanimelist.net/images/anime/1948/103308t.jpg',
   'large_image_url': 'https://cdn.myanimelist.net/images/anime/1948/103308l.jpg'},
  'webp': {'image_url': 'https://cdn.myanimelist.net/images/anime/1948/103308.webp',
   'small_image_url': 'https://cdn.myanimelist.net/images/anime/1948/103308t.webp',
   'large_image_url': 'https://cdn.myanimelist.net/images/anime/1948/103308l.webp'}},
 'trailer': {'youtube_id': '5W5tvt1WhMo',
  'url': 'https://www.youtube.com/watch?v=5W5tvt1WhMo',
  'embed_url': 'https://www.youtube.com/embed/5W5tvt1WhMo?enablejsapi=1&wmode=opaque&autoplay=1',
  'images': {'image_url': 'https://img.youtube.com/vi/5W5tvt1WhMo/default.jpg',
   'small_image_url': 'https://img.youtube.com/vi/5W5tvt1WhMo/sddefault.jpg',
   'medium_image_url': 'ht

In [9]:
# Convert full list of shows to DataFrame
df = pd.DataFrame(full_anime_list)
df.head(4)

,mal_id,url,images,trailer,approved,titles,title,title_english,title_japanese,title_synonyms,...,season,year,broadcast,producers,licensors,studios,genres,explicit_genres,themes,demographics
0,48820,https://myanimelist.net/anime/48820/Mahou_Shou...,{'jpg': {'image_url': 'https://cdn.myanimelist...,"{'youtube_id': 'vSdolbHyvkA', 'url': 'https://...",True,"[{'type': 'Default', 'title': 'Mahou Shoujo Ma...",Mahou Shoujo Madoka★Magica Movie 4: Walpurgis ...,Puella Magi Madoka Magica the Movie - Walpurgi...,劇場版 魔法少女まどか☆マギカ〈ワルプルギスの廻天〉,"[Mahou Shoujo Madoka Magika Movie 4, Magical G...",...,None,NaN,"{'day': None, 'time': None, 'timezone': None, ...","[{'mal_id': 17, 'type': 'anime', 'name': 'Anip...",[],"[{'mal_id': 44, 'type': 'anime', 'name': 'Shaf...","[{'mal_id': 8, 'type': 'anime', 'name': 'Drama...",[],"[{'mal_id': 66, 'type': 'anime', 'name': 'Maho...",[]
1,57616,https://myanimelist.net/anime/57616/Kimi_no_Ko...,{'jpg': {'image_url': 'https://cdn.myanimelist...,"{'youtube_id': 's7Sm7hA6Ntg', 'url': 'https://...",True,"[{'type': 'Default', 'title': 'Kimi no Koto ga...",Kimi no Koto ga Daidaidaidaidaisuki na 100-nin...,"The 100 Girlfriends Who Really, Really, Really...",君のことが大大大大大好きな100人の彼女 2期,[Hyakkano 2nd Season],...,winter,2025.0,"{'day': None, 'time': None, 'timezone': None, ...","[{'mal_id': 104, 'type': 'anime', 'name': 'Lan...",[],"[{'mal_id': 1722, 'type': 'anime', 'name': 'Bi...","[{'mal_id': 4, 'type': 'anime', 'name': 'Comed...",[],"[{'mal_id': 35, 'type': 'anime', 'name': 'Hare...","[{'mal_id': 42, 'type': 'anime', 'name': 'Sein..."
2,58939,https://myanimelist.net/anime/58939/Sakamoto_Days,{'jpg': {'image_url': 'https://cdn.myanimelist...,"{'youtube_id': 'XA4jmyg3rhY', 'url': 'https://...",True,"[{'type': 'Default', 'title': 'Sakamoto Days'}...",Sakamoto Days,None,SAKAMOTO DAYS,[],...,winter,2025.0,"{'day': None, 'time': None, 'timezone': None, ...",[],[],"[{'mal_id': 73, 'type': 'anime', 'name': 'TMS ...","[{'mal_id': 1, 'type': 'anime', 'name': 'Actio...",[],[],"[{'mal_id': 27, 'type': 'anime', 'name': 'Shou..."
3,53924,https://myanimelist.net/anime/53924/Jibaku_Sho...,{'jpg': {'image_url': 'https://cdn.myanimelist...,"{'youtube_id': 'tTj-zHKzwXY', 'url': 'https://...",True,"[{'type': 'Default', 'title': 'Jibaku Shounen ...",Jibaku Shounen Hanako-kun 2,Toilet-Bound Hanako-kun Season 2,地縛少年花子くん2,[],...,winter,2025.0,"{'day': 'Sundays', 'time': '16:30', 'timezone'...","[{'mal_id': 58, 'type': 'anime', 'name': 'Squa...",[],"[{'mal_id': 456, 'type': 'anime', 'name': 'Ler...","[{'mal_id': 37, 'type': 'anime', 'name': 'Supe...",[],"[{'mal_id': 23, 'type': 'anime', 'name': 'Scho...","[{'mal_id': 27, 'type': 'anime', 'name': 'Shou..."


In [149]:
# Filter by relevant columns
df_mal = df[[
    'mal_id',
    'url',
    'images',
    'title',
    'type',
    'source',
    'aired',
    'episodes',
    'duration',
    'rating',
    'score',
    'scored_by',
    'rank',
    'members',
    'popularity',
    'favorites',
    'season',
    'year',
    'studios',
    'genres',
    'themes',
    'demographics'
]]

### 2. Genres:
- MAL categorizes shows by three types of genre-like categories: Genres, Themes, and Demographics.
- We will combine them into one categorizer and use `sklearn.preprocessing.MultiLabelBinarizer` to create a column for each genre and encode into a one-hot encoded format. Doing this will allow for easier counts of genres, etc.
- The `genre_list` is retrieved from the API.

In [15]:
GENRE_LIST_ENDPOINT = f'{BASE_URL}/genres/anime'

In [16]:
# Retrieve genres from API
response = requests.get(GENRE_LIST_ENDPOINT)
genres = response.json()
genres

{'data': [{'mal_id': 1,
   'name': 'Action',
   'url': 'https://myanimelist.net/anime/genre/1/Action',
   'count': 5235},
  {'mal_id': 2,
   'name': 'Adventure',
   'url': 'https://myanimelist.net/anime/genre/2/Adventure',
   'count': 4163},
  {'mal_id': 5,
   'name': 'Avant Garde',
   'url': 'https://myanimelist.net/anime/genre/5/Avant_Garde',
   'count': 928},
  {'mal_id': 46,
   'name': 'Award Winning',
   'url': 'https://myanimelist.net/anime/genre/46/Award_Winning',
   'count': 253},
  {'mal_id': 28,
   'name': 'Boys Love',
   'url': 'https://myanimelist.net/anime/genre/28/Boys_Love',
   'count': 181},
  {'mal_id': 4,
   'name': 'Comedy',
   'url': 'https://myanimelist.net/anime/genre/4/Comedy',
   'count': 7445},
  {'mal_id': 8,
   'name': 'Drama',
   'url': 'https://myanimelist.net/anime/genre/8/Drama',
   'count': 2963},
  {'mal_id': 10,
   'name': 'Fantasy',
   'url': 'https://myanimelist.net/anime/genre/10/Fantasy',
   'count': 5935},
  {'mal_id': 26,
   'name': 'Girls Love',

In [17]:
# Transform genres into list
genre_list = [genre['name'] for genre in genres['data']]
genre_list

['Action',
 'Adventure',
 'Avant Garde',
 'Award Winning',
 'Boys Love',
 'Comedy',
 'Drama',
 'Fantasy',
 'Girls Love',
 'Gourmet',
 'Horror',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Slice of Life',
 'Sports',
 'Supernatural',
 'Suspense',
 'Ecchi',
 'Erotica',
 'Hentai',
 'Adult Cast',
 'Anthropomorphic',
 'CGDCT',
 'Childcare',
 'Combat Sports',
 'Crossdressing',
 'Delinquents',
 'Detective',
 'Educational',
 'Gag Humor',
 'Gore',
 'Harem',
 'High Stakes Game',
 'Historical',
 'Idols (Female)',
 'Idols (Male)',
 'Isekai',
 'Iyashikei',
 'Love Polygon',
 'Magical Sex Shift',
 'Mahou Shoujo',
 'Martial Arts',
 'Mecha',
 'Medical',
 'Military',
 'Music',
 'Mythology',
 'Organized Crime',
 'Otaku Culture',
 'Parody',
 'Performing Arts',
 'Pets',
 'Psychological',
 'Racing',
 'Reincarnation',
 'Reverse Harem',
 'Romantic Subtext',
 'Samurai',
 'School',
 'Showbiz',
 'Space',
 'Strategy Game',
 'Super Power',
 'Survival',
 'Team Sports',
 'Time Travel',
 'Vampire',
 'Video Game',
 'Visual Ar

In [153]:
# Combines genres, themes, and demographics columns
def combine_genre_cols(row):
    return row['genres'] + row['themes'] + row['demographics']

# Converts genre dictionaries into list of genre names
def extract_genre_names(genres_in_show):
    return [genre['name'] for genre in genres_in_show]

df_mal = df_mal.copy()
df_mal['genres_combined'] = df_mal.apply(combine_genre_cols, axis=1).apply(extract_genre_names)
df_mal = df_mal.drop(columns=['genres', 'themes', 'demographics'])
df_mal

,mal_id,url,images,title,type,source,aired,episodes,duration,rating,score,scored_by,rank,members,popularity,favorites,season,year,studios,genres_combined
0,48820,https://myanimelist.net/anime/48820/Mahou_Shou...,{'jpg': {'image_url': 'https://cdn.myanimelist...,Mahou Shoujo Madoka★Magica Movie 4: Walpurgis ...,Movie,Original,"{'from': '2025-01-01T00:00:00+00:00', 'to': No...",1.0,Unknown,PG-13 - Teens 13 or older,NaN,NaN,NaN,65825,2772,339,None,NaN,"[{'mal_id': 44, 'type': 'anime', 'name': 'Shaf...","[Drama, Fantasy, Suspense, Mahou Shoujo, Psych..."
1,57616,https://myanimelist.net/anime/57616/Kimi_no_Ko...,{'jpg': {'image_url': 'https://cdn.myanimelist...,Kimi no Koto ga Daidaidaidaidaisuki na 100-nin...,TV,Manga,"{'from': '2025-01-01T00:00:00+00:00', 'to': No...",NaN,Unknown,PG-13 - Teens 13 or older,NaN,NaN,NaN,42413,3515,269,winter,2025.0,"[{'mal_id': 1722, 'type': 'anime', 'name': 'Bi...","[Comedy, Romance, Harem, Parody, School, Seinen]"
2,58939,https://myanimelist.net/anime/58939/Sakamoto_Days,{'jpg': {'image_url': 'https://cdn.myanimelist...,Sakamoto Days,TV,Manga,"{'from': '2025-01-01T00:00:00+00:00', 'to': No...",NaN,Unknown,None,NaN,NaN,NaN,35001,3858,156,winter,2025.0,"[{'mal_id': 73, 'type': 'anime', 'name': 'TMS ...","[Action, Comedy, Shounen]"
3,53924,https://myanimelist.net/anime/53924/Jibaku_Sho...,{'jpg': {'image_url': 'https://cdn.myanimelist...,Jibaku Shounen Hanako-kun 2,TV,Manga,"{'from': '2025-01-01T00:00:00+00:00', 'to': No...",NaN,Unknown,PG-13 - Teens 13 or older,NaN,NaN,NaN,25462,4491,142,winter,2025.0,"[{'mal_id': 456, 'type': 'anime', 'name': 'Ler...","[Supernatural, School, Shounen]"
4,59490,https://myanimelist.net/anime/59490/Kaijuu_8-g...,{'jpg': {'image_url': 'https://cdn.myanimelist...,Kaijuu 8-gou: Hoshina no Kyuujitsu,Special,Manga,"{'from': '2025-01-01T00:00:00+00:00', 'to': No...",1.0,Unknown,PG-13 - Teens 13 or older,NaN,NaN,NaN,19728,5008,59,None,NaN,[],"[Adult Cast, Shounen]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25375,50577,https://myanimelist.net/anime/50577/Imokawa_Mu...,{'jpg': {'image_url': 'https://cdn.myanimelist...,Imokawa Mukuzo: Tsuri no Maki,Movie,Original,"{'from': '1917-09-09T00:00:00+00:00', 'to': No...",1.0,Unknown,G - All Ages,NaN,NaN,18135.0,195,20871,0,None,NaN,[],[Comedy]
25376,10757,https://myanimelist.net/anime/10757/Kachikachi...,{'jpg': {'image_url': 'https://cdn.myanimelist...,Kachikachi Yama,Movie,Other,"{'from': '1917-10-20T00:00:00+00:00', 'to': No...",1.0,8 min,G - All Ages,5.32,237.0,12132.0,862,14347,2,None,NaN,[],[Drama]
25377,10756,https://myanimelist.net/anime/10756/Shita-kiri...,{'jpg': {'image_url': 'https://cdn.myanimelist...,Shita-kiri Suzume,Movie,Original,"{'from': '1917-10-18T00:00:00+00:00', 'to': No...",1.0,Unknown,G - All Ages,NaN,NaN,13695.0,644,15343,1,None,NaN,[],[Drama]
25378,23191,https://myanimelist.net/anime/23191/Chokin_no_...,{'jpg': {'image_url': 'https://cdn.myanimelist...,Chokin no Susume,Movie,Original,"{'from': '1917-10-07T00:00:00+00:00', 'to': No...",1.0,Unknown,G - All Ages,NaN,NaN,16633.0,592,15695,0,None,NaN,[],[Comedy]


In [154]:
# One hot encode genre data into DataFrame
mlb = MultiLabelBinarizer()
encoded_array = mlb.fit_transform(df_mal['genres_combined'])
df_encoded = pd.DataFrame(encoded_array, columns=mlb.classes_)

In [155]:
# Concat with original DataFrame
df_mal_encoded = pd.concat([df_mal, df_encoded], axis=1).drop(columns=['genres_combined'])
df_mal_encoded.head(7)

,mal_id,url,images,title,type,source,aired,episodes,duration,rating,...,Super Power,Supernatural,Survival,Suspense,Team Sports,Time Travel,Vampire,Video Game,Visual Arts,Workplace
0,48820,https://myanimelist.net/anime/48820/Mahou_Shou...,{'jpg': {'image_url': 'https://cdn.myanimelist...,Mahou Shoujo Madoka★Magica Movie 4: Walpurgis ...,Movie,Original,"{'from': '2025-01-01T00:00:00+00:00', 'to': No...",1.0,Unknown,PG-13 - Teens 13 or older,...,0,0,0,1,0,0,0,0,0,0
1,57616,https://myanimelist.net/anime/57616/Kimi_no_Ko...,{'jpg': {'image_url': 'https://cdn.myanimelist...,Kimi no Koto ga Daidaidaidaidaisuki na 100-nin...,TV,Manga,"{'from': '2025-01-01T00:00:00+00:00', 'to': No...",NaN,Unknown,PG-13 - Teens 13 or older,...,0,0,0,0,0,0,0,0,0,0
2,58939,https://myanimelist.net/anime/58939/Sakamoto_Days,{'jpg': {'image_url': 'https://cdn.myanimelist...,Sakamoto Days,TV,Manga,"{'from': '2025-01-01T00:00:00+00:00', 'to': No...",NaN,Unknown,None,...,0,0,0,0,0,0,0,0,0,0
3,53924,https://myanimelist.net/anime/53924/Jibaku_Sho...,{'jpg': {'image_url': 'https://cdn.myanimelist...,Jibaku Shounen Hanako-kun 2,TV,Manga,"{'from': '2025-01-01T00:00:00+00:00', 'to': No...",NaN,Unknown,PG-13 - Teens 13 or older,...,0,1,0,0,0,0,0,0,0,0
4,59490,https://myanimelist.net/anime/59490/Kaijuu_8-g...,{'jpg': {'image_url': 'https://cdn.myanimelist...,Kaijuu 8-gou: Hoshina no Kyuujitsu,Special,Manga,"{'from': '2025-01-01T00:00:00+00:00', 'to': No...",1.0,Unknown,PG-13 - Teens 13 or older,...,0,0,0,0,0,0,0,0,0,0
5,59489,https://myanimelist.net/anime/59489/Kaijuu_8-g...,{'jpg': {'image_url': 'https://cdn.myanimelist...,Kaijuu 8-gou Movie,Movie,Manga,"{'from': '2025-01-01T00:00:00+00:00', 'to': No...",1.0,Unknown,PG-13 - Teens 13 or older,...,0,0,0,0,0,0,0,0,0,0
6,59654,https://myanimelist.net/anime/59654/Jujutsu_Ka...,{'jpg': {'image_url': 'https://cdn.myanimelist...,Jujutsu Kaisen: Kaigyoku/Gyokusetsu,Movie,Manga,"{'from': '2025-01-01T00:00:00+00:00', 'to': No...",1.0,Unknown,R - 17+ (violence & profanity),...,0,0,0,0,0,0,0,0,0,0


### 3. Convert `studio` Names from `dict` to `string`:

In [156]:
# Extract studio names and convert to string
def extract_studio_names(studio_dicts):
    studio_names_list = [studio['name'] for studio in studio_dicts]
    studio_names = ", ".join(studio_names_list)
    return studio_names

In [157]:
studios_extracted = df_mal_encoded['studios'].apply(extract_studio_names)

In [158]:
studios_extracted

0                           Shaft
1        Bibury Animation Studios
2               TMS Entertainment
3                          Lerche
4                                
                   ...           
25375                            
25376                            
25377                            
25378                            
25379                            
Name: studios, Length: 25380, dtype: object

In [159]:
df_mal_encoded['studios'] = studios_extracted

### 4. Extract Large Image URL from `images`:

In [160]:
# Extract large jpg url from 'images' column
def extract_jpg_url(image_url_dict):
    return image_url_dict['jpg']['large_image_url']

jpg_urls = df_mal['images'].apply(extract_jpg_url)

In [161]:
df_mal_encoded['images'] = jpg_urls

### Add `season` and `year` for Non-TV Entries:
- Non-TV entries have `null` values for `season` and `year`.
- We will use the airing date to assign the right values to these columns.

In [192]:
# Checking output of 'aired' column
df_mal_encoded['aired'][3997]

{'from': '2021-12-24T00:00:00+00:00',
 'to': None,
 'prop': {'from': {'day': 24, 'month': 12, 'year': 2021},
  'to': {'day': None, 'month': None, 'year': None}},
 'string': 'Dec 24, 2021'}

In [368]:
def get_season_year(row):
    if pd.isna(row['season']) and pd.isna(row['year']):
        date = datetime.fromisoformat(row['aired']['from'])
        row['year'] = date.year
        if date.month in [1, 2, 3]:
            row['season'] = 'Winter'
        if date.month in [4, 5, 6]:
            row['season'] = 'Spring'
        if date.month in [7, 8, 9]:
            row['season'] = 'Summer'
        if date.month in [10, 11, 12]:
            row['season'] = 'Fall'
    return row
df_mal_filled = df_mal_encoded.apply(get_season_year, axis=1)

### 5. Extract `aired_dates` From `aired` Column:

In [369]:
def extract_aired_dates(aired_dict):
    aired_date = aired_dict['from']
    aired_datetime = datetime.fromisoformat(aired_date)
    return aired_datetime.strftime('%b ') + str(aired_datetime.day) + aired_datetime.strftime(', %Y')

In [370]:
aired_dates = df_mal_filled['aired'].apply(extract_aired_dates)
df_mal_filled['aired'] = aired_dates

### 6. Capitalize `season`, then Combine `season` and `year` Columns:

In [371]:
df_mal_capitalized = df_mal_filled.copy()
df_mal_capitalized['season'] = df_mal_filled['season'].str.capitalize()

In [372]:
df_mal_capitalized['season_year'] = (df_mal_capitalized['season'] + ' ' + df_mal_capitalized['year'].astype(str).str.replace(r'\.0', '', regex=True)).replace(np.nan, None)

### 7. Check Duplicates:

In [373]:
df_mal_capitalized[df_mal_capitalized.duplicated()]

,mal_id,url,images,title,type,source,aired,episodes,duration,rating,...,Supernatural,Survival,Suspense,Team Sports,Time Travel,Vampire,Video Game,Visual Arts,Workplace,season_year
7,59654,https://myanimelist.net/anime/59654/Jujutsu_Ka...,https://cdn.myanimelist.net/images/anime/1820/...,Jujutsu Kaisen: Kaigyoku/Gyokusetsu,Movie,Manga,"Jan 1, 2025",1.0,Unknown,R - 17+ (violence & profanity),...,0,0,0,0,0,0,0,0,0,Winter 2025
10,59435,https://myanimelist.net/anime/59435/Undead_Unl...,https://cdn.myanimelist.net/images/anime/1964/...,Undead Unluck (Special),Special,Manga,"Jan 1, 2025",1.0,1 hr,R - 17+ (violence & profanity),...,0,0,0,0,0,0,0,0,0,Winter 2025
19,59408,https://myanimelist.net/anime/59408/Mononoke_M...,https://cdn.myanimelist.net/images/anime/1475/...,Mononoke Movie: Dai ni Shou - Hinezumi,Movie,Original,"Mar 14, 2025",NaN,Unknown,R - 17+ (violence & profanity),...,1,0,1,0,0,0,0,0,0,Winter 2025
29,59636,https://myanimelist.net/anime/59636/Uma_Musume...,https://cdn.myanimelist.net/images/anime/1621/...,Uma Musume: Cinderella Gray,TV,Manga,"Jan 1, 2025",NaN,Unknown,None,...,0,0,0,0,0,0,0,0,0,Winter 2025
39,59406,https://myanimelist.net/anime/59406/Binan_Kouk...,https://cdn.myanimelist.net/images/anime/1737/...,Binan Koukou Chikyuu Bouei-bu Eternal Love!,Movie,Original,"Jan 1, 2025",1.0,Unknown,None,...,0,0,0,0,0,0,0,0,0,Winter 2025
547,59493,https://myanimelist.net/anime/59493/Tensei_shi...,https://cdn.myanimelist.net/images/anime/1465/...,Tensei shitara Slime Datta Ken: Kanwa - Lumino...,TV Special,Manga,"Aug 10, 2024",1.0,24 min,PG-13 - Teens 13 or older,...,0,0,0,0,0,0,0,0,0,Summer 2024
557,59497,https://myanimelist.net/anime/59497/Rising_Imp...,https://cdn.myanimelist.net/images/anime/1161/...,Rising Impact Season 2,ONA,Manga,"Aug 6, 2024",14.0,26 min per ep,PG-13 - Teens 13 or older,...,0,0,0,0,0,0,0,0,0,Summer 2024
560,59390,https://myanimelist.net/anime/59390/Terminator_0,https://cdn.myanimelist.net/images/anime/1655/...,Terminator 0,ONA,Other,"Aug 29, 2024",8.0,27 min per ep,R - 17+ (violence & profanity),...,0,0,0,0,1,0,0,0,0,Summer 2024
568,56207,https://myanimelist.net/anime/56207/Quanzhi_Fa...,https://cdn.myanimelist.net/images/anime/1516/...,Quanzhi Fashi Tebie Pian: Shenmi Weituo,ONA,Web novel,"Aug 16, 2024",6.0,22 min per ep,PG-13 - Teens 13 or older,...,0,0,0,0,0,0,0,0,0,Summer 2024
579,59465,https://myanimelist.net/anime/59465/Fate_Grand...,https://cdn.myanimelist.net/images/anime/1412/...,Fate/Grand Order: Fujimaru Ritsuka wa Wakarana...,ONA,Manga,"Aug 3, 2024",NaN,2 min,PG-13 - Teens 13 or older,...,0,0,0,0,0,0,0,0,0,Summer 2024


In [374]:
df_mal_capitalized = df_mal_capitalized.drop_duplicates()

### 8. Export to .json():

In [380]:
df_mal_capitalized.to_csv('df_mal_cleaned.csv')